# Web scraping con Python: primi passi

## Introduzione

- note preliminari: pacchetti requests e beautifulsoup4 (pip install...)
- scraping da pagine html semplici
- scraping da risultati distribuiti su molte pagine

DISCLAIMER: i proprietari dei siti potrebbero avere qualcosa da ridire! 

Date un'occhiata al file robots.txt nella "radice" di ogni sito.

Per esempio:
![title](robotstxt.png)

## Scraping di una pagina semplice

https://it.wikipedia.org/wiki/Categoria:Cantanti_trap

![title](trappers.png)

Perché non fare direttamente copia-incolla?

1) così impariamo le basi del web scraping :)

2) il risultato è una lista di testo semplice, pronta per essere salvata o manipolata con python.

Pacchetti necessari:

In [1]:
import requests # to make http call and donwload html sources; settings:

headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'

from bs4 import BeautifulSoup as bs # to navigate the downloaded html

import re # regular expressions in python

import random # generate random integers

import time # used to wait some time between http requests

Definiamo il nostro url e facciamo una chiamata http:

In [2]:
url = "https://it.wikipedia.org/wiki/Categoria:Cantanti_trap" # a string

r = requests.get(url, headers = headers) # using the headers specified above

print(r)

<Response [200]>


Response 200 significa che è tutto OK.

Altre risposte (meno OK) possono essere 404 "Not Found" and 503 "Service Unavailable".

L'oggetto response ha un metodo text:

In [3]:
print(r.text)

<!DOCTYPE html>
<html class="client-nojs" lang="it" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Categoria:Cantanti trap - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Categoria:Cantanti_trap","wgTitle":"Cantanti trap","wgCurRevisionId":101188935,"wgRevisionId":101188935,"wgArticleId":5439423,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Cantanti hip hop","Musicisti trap"],"wgBreakFrames":false,"wgPageContentLanguage":"it","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","gennaio","febbraio","marzo","aprile","maggio","giugno",

Per lavorare con questa zuppa, usiamo beautifulsoup:

In [4]:
soup = bs(r.text, "lxml") # soupify i.e. make tag soup easy to navigate, using the lxml parser

In [5]:
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="it">
<head>
<meta charset="utf-8"/>
<title>Categoria:Cantanti trap - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Categoria:Cantanti_trap","wgTitle":"Cantanti trap","wgCurRevisionId":101188935,"wgRevisionId":101188935,"wgArticleId":5439423,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Cantanti hip hop","Musicisti trap"],"wgBreakFrames":false,"wgPageContentLanguage":"it","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","gennaio","febbraio","marzo","aprile","maggio","giugno",

Ma è identico!

In realtà, no:

1) il primo è puro testo, ad esempio manipolabile con le regex per estrarre informazioni utili

2) il secondo sembra testo, in realtà è un oggetto con una struttura navigabile tramite tag html.

Per esempio, se vogliamo estrarre il titolo della pagina:

In [6]:
soup.find('title') # find method yield the first found occurrence

<title>Categoria:Cantanti trap - Wikipedia</title>

Come facciamo a sapere dove sono le informazioni che ci interessano?

Ispezioniamo la sorgente (source) della pagina!

(su Google Chrome, ctrl+shift+I apre il source inspector)

![title](inspect.png)

Zoom in:

![title](inspect_zoom.png)

I nomi sono nei tag *a*.

il metodo *findAll* trova tutti i tag di un certo tipo nella zuppa:

In [7]:
a_tags = soup.findAll('a') # findAll output is a list

print(a_tags)

[<a id="top"></a>, <a class="mw-helplink" href="/wiki/Aiuto:Categorie" target="_blank">Aiuto</a>, <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>, <a class="mw-jump-link" href="#p-search">Jump to search</a>, <a href="/wiki/Cantante" title="Cantante">cantanti</a>, <a href="/wiki/Trap_(genere_musicale)" title="Trap (genere musicale)">trap</a>, <a href="/wiki/Categoria:Cantanti_per_nazionalit%C3%A0" title="Categoria:Cantanti per nazionalità">Cantanti per nazionalità</a>, <a href="/wiki/Categoria:Cantanti_per_genere" title="Categoria:Cantanti per genere">Cantanti per genere (tutti)</a>, <a href="/wiki/Categoria:Gruppi_musicali_trap" title="Categoria:Gruppi musicali trap">Gruppi musicali trap</a>, <a href="/wiki/Categoria:Musicisti_trap" title="Categoria:Musicisti trap">Musicisti trap</a>, <a href="/wiki/Categoria:Disc_jockey_trap" title="Categoria:Disc jockey trap">Disc jockey trap</a>, <a href="/wiki/Categoria:Album_trap" title="Categoria:Album trap">Album trap</a>, <a href

Il risultato è molto lontano dall'essere leggibile e pulito.

1) contiene tag *a* che non ci interessano

2) contiene molte informazioni in più rispetto ai soli nomi

Torniamo alla sorgente:
![title](inspect0.png)

Zoom in:
![title](inspect0_zoom.png)

Estraiamo prima il *div* con classe "mw-category-generated", poi estraiamo i tag *a* solo da quel *div*:

In [8]:
div = soup.find('div', {'class' : 'mw-category-generated'}) 
# notice: the second argument is a dictionary, so it can potentially have more entries, for more refined definitions

In [9]:
print(div)

<div class="mw-category-generated" dir="ltr" lang="it"><div id="mw-pages">
<h2>Pagine nella categoria "Cantanti trap"</h2>
<p>Questa categoria contiene le 85 pagine indicate di seguito, su un totale di 85.
</p><div class="mw-content-ltr" dir="ltr" lang="it"><div class="mw-category"><div class="mw-category-group"><h3>0–9</h3>
<ul><li><a href="/wiki/6ix9ine" title="6ix9ine">6ix9ine</a></li>
<li><a href="/wiki/21_Savage" title="21 Savage">21 Savage</a></li></ul></div><div class="mw-category-group"><h3>A</h3>
<ul><li><a href="/wiki/Anuel_AA" title="Anuel AA">Anuel AA</a></li></ul></div><div class="mw-category-group"><h3>B</h3>
<ul><li><a href="/wiki/Bad_Bunny" title="Bad Bunny">Bad Bunny</a></li>
<li><a href="/wiki/Bhad_Bhabie" title="Bhad Bhabie">Bhad Bhabie</a></li>
<li><a href="/wiki/Birdman_(rapper)" title="Birdman (rapper)">Birdman (rapper)</a></li>
<li><a href="/wiki/Booba" title="Booba">Booba</a></li>
<li><a href="/wiki/Quando_Rondo" title="Quando Rondo">Quando Rondo</a></li></ul></

In [10]:
a_tags = div.findAll('a')

print(a_tags[0:5])

[<a href="/wiki/6ix9ine" title="6ix9ine">6ix9ine</a>, <a href="/wiki/21_Savage" title="21 Savage">21 Savage</a>, <a href="/wiki/Anuel_AA" title="Anuel AA">Anuel AA</a>, <a href="/wiki/Bad_Bunny" title="Bad Bunny">Bad Bunny</a>, <a href="/wiki/Bhad_Bhabie" title="Bhad Bhabie">Bhad Bhabie</a>]


Per estrarre solo i nomi, estraiamo l'elemento text dai nostri tag *a*:

In [11]:
names = [a.text for a in a_tags]

print(names)

['6ix9ine', '21 Savage', 'Anuel AA', 'Bad Bunny', 'Bhad Bhabie', 'Birdman (rapper)', 'Booba', 'Quando Rondo', 'Cardi B', 'Chanmina', 'Chief Keef', 'CL (cantante)', 'Dej Loaf', 'DJ Paul', 'Duke Montana', 'Famous Dex', 'Fetty Wap', 'Frauenarzt', 'Future (rapper)', 'Ghali (rapper)', 'Gradur', 'Gucci Mane', 'Haftbefehl', 'Ace Hood', 'Izi (rapper)', 'Jeezy', 'Juice Wrld', 'Juicy J', 'Kaaris', 'Keith Ape', 'Ketama126', 'Wiz Khalifa', 'Kid Kaze', 'Lacrim', 'Laïoung', 'Achille Lauro (rapper)', 'Lazza', 'Lil B', 'Lil Baby', 'Lil Pump', 'Lil Skies', 'Lil Uzi Vert', 'Lil Wayne', 'Lil Yachty', 'Lord Infamous', 'MadeinTYO', 'Meek Mill', 'Mino (rapper)', 'Stella Mwangi', 'Niska', 'O.T. Genasis', 'Offset (rapper)', 'Ozuna', 'PartyNextDoor', 'Project Pat', 'Quavo', 'Alemán', 'Lil Reese', 'Rich the Kid', 'Dawn Richard', 'Rick Ross', 'Robb Banks', 'Rvssian', 'SCH (rapper)', 'SD (rapper)', 'Desiigner', 'Shawty Lo', 'Sheck Wes', 'Jaden Smith', 'Smokepurpp', 'Soulja Boy', 'Ronny J', 'T.I.', 'Takeoff', 'Tay

Alla fine possiamo salvare la lista come file di testo:

In [12]:
with open('trappers.txt', 'w+') as output_file:
    for name in names:
        output_file.write(name+'\n')        

In [13]:
# bonus: text is not the only available element; suppose we need urls:

urls = [a.attrs['href'] for a in a_tags]

print(urls[0:5])

['/wiki/6ix9ine', '/wiki/21_Savage', '/wiki/Anuel_AA', '/wiki/Bad_Bunny', '/wiki/Bhad_Bhabie']


## Scraping di risultati su più pagine

![title](hotels.png)

In ogni regione i risultati sono distributiti su più pagine:

![title](abruzzo0.png)

Idea: visitiamo ogni regione, scopriamo quante pagine ci sono in quella regione, scarichiamo tutti i nomi in tutte le pagine. 

Iniziamo definendo un template di url e una lista di regioni con cui completarlo:

In [14]:
# curly braces will be replaced with region name using format
region_template_url = "http://www.elenco-alberghi.it/{}/alberghi-hotels.asp"

# manually taken from homepage of www.elenco-alberghi.it
regions = ["abruzzo", "basilicata", "calabria", "campania", "emilia-romagna", "friuli-venezia-giulia",
"lazio", "liguria", "lombardia", "marche", "molise", "piemonte", "puglia", "sardegna", "sicilia", "toscana",
"trentino-alto-adige", "umbria", "valle-d-aosta", "veneto"]

Per ogni regione, cerchiamo il numero massimo di pagine nella sorgente:

![title](pagination.png)

Zoom in:

![title](pag_zoom.png)

I tag *li* nel *div* "paginazione" contengono i numeri di pagina sia nell'elemento url sia in quello testo.

Ma non nell'ultima pagina, il cui elemento testo è "...". 
    
Estraiamo quindi il numero di pagine dall'elemento url dell'ultimo *li* (in questo caso 81):

Prima di tutto completiamo l'url:

In [15]:
region = "abruzzo" # for example
reg_url = region_template_url.format(region) # insert region into template url

print(reg_url)

http://www.elenco-alberghi.it/abruzzo/alberghi-hotels.asp


Facciamo la chiamata http e facciamo il parsing con bs:

In [16]:
reg_r = requests.get(reg_url, headers=headers) # http request to page
print(reg_r)

reg_soup = bs(reg_r.text, "lxml")  # soupify

<Response [200]>


Estraiamo i tag *li* dal *div* "paginazione":

In [17]:
div = reg_soup.find('div', {'id' : 'paginazione'}) # pages of numbers are in this div
    
li_tags = [li for li in div.findAll('li')] # pages numbers are in these li elements

print(li_tags)

[<li id="inactive">Pagine:</li>, <li id="activelink"><a href="javascript:void(0)">1</a></li>, <li><a href="/abruzzo/alberghi-hotels_2.asp" title="Vai alla pagina n. 2">2</a></li>, <li><a href="/abruzzo/alberghi-hotels_3.asp" title="Vai alla pagina n. 3">3</a></li>, <li><a href="/abruzzo/alberghi-hotels_4.asp" title="Vai alla pagina n. 4">4</a></li>, <li><a href="/abruzzo/alberghi-hotels_5.asp" title="Vai alla pagina n. 5">5</a></li>, <li><a href="/abruzzo/alberghi-hotels_6.asp" title="Vai alla pagina n. 6">6</a></li>, <li><a href="/abruzzo/alberghi-hotels_7.asp" title="Vai alla pagina n. 7">7</a></li>, <li><a href="/abruzzo/alberghi-hotels_8.asp" title="Vai alla pagina n. 8">8</a></li>, <li><a href="/abruzzo/alberghi-hotels_9.asp" title="Vai alla pagina n. 9">9</a></li>, <li><a href="/abruzzo/alberghi-hotels_10.asp" title="Vai alla pagina n. 10">10</a></li>, <li><a href="/abruzzo/alberghi-hotels_81.asp" title="Vai all'ultima pagina">...</a></li>]


Ci serve l'url dell'ultimo *li*:

In [18]:
last_li = li_tags[-1] # we want the last li

print(last_li)

<li><a href="/abruzzo/alberghi-hotels_81.asp" title="Vai all'ultima pagina">...</a></li>


In [19]:
last_li_url = last_li.find('a').attrs['href'] 
print(last_li_url)

/abruzzo/alberghi-hotels_81.asp


Infine estraiamo la cifra e convertiamo a numero intero:

In [20]:
N = int(re.findall(r'\d+', last_li_url)[0]) # convert to integer the only digit found
print(N)

81


Per rendere la procedura riusabile, definiamo una funzione:

In [21]:
def howmany_pages(region): # region as input
    reg_url = region_template_url.format(region) # complete url
    reg_r = requests.get(reg_url, headers=headers) # http request
    reg_soup = bs(reg_r.text, "lxml")  # soupify    
    div = reg_soup.find('div', {'id' : 'paginazione'}) # extract div    
    li_tags = [li for li in div.findAll('li')] # extract li tags    
    last_li = li_tags[-1].find('a').attrs['href'] # url of last li    
    N = int(re.findall(r'\d+', last_li)[0]) # to integer
    
    return(N)

In [22]:
howmany_pages("abruzzo")

81

In [23]:
howmany_pages("piemonte")

149

Applichiamo la funzione alla lista di regioni, creando un dizionario:

In [24]:
# collect in a dictionary the name of each region with its max number of pages
regions_dic = {region : howmany_pages(region) for region in regions}

In [25]:
print(regions_dic)

{'sardegna': 97, 'toscana': 406, 'friuli-venezia-giulia': 65, 'emilia-romagna': 379, 'veneto': 288, 'calabria': 73, 'valle-d-aosta': 36, 'basilicata': 17, 'sicilia': 183, 'campania': 174, 'liguria': 123, 'piemonte': 149, 'lombardia': 256, 'umbria': 74, 'lazio': 184, 'trentino-alto-adige': 300, 'abruzzo': 81, 'puglia': 108, 'marche': 103, 'molise': 8}


Ora passiamo al download dei nomi, da ogni pagina di ogni regione.

Per esempio, la settima pagina della region Marche:
http://www.elenco-alberghi.it/marche/alberghi-hotels_7.asp
![title](marche7.png)

Zoom in:

![title](marche7_zoom.png)

Vogliamo estrarre l'elemento testo dagli *a* dello *span* con classe "titololista":

In [26]:
# we start from a url template
page_template_url = "http://www.elenco-alberghi.it/{}/alberghi-hotels_{}.asp"

# second curly braces will be the page number

In [27]:
region = "marche" # set region manually
i = 7 # set page number manually

In [28]:
reg_url = page_template_url.format(region, i) # fill the details of the url
print(reg_url)

http://www.elenco-alberghi.it/marche/alberghi-hotels_7.asp


In [29]:
reg_r = requests.get(reg_url, headers = headers) # http call
reg_soup = bs(reg_r.text, "lxml")  # soupify

In [30]:
span = reg_soup.findAll('span', {'class' : 'titololista'}) # get the span
print(span)

[<span class="titololista"><a href="http://www.elenco-alberghi.it/marche/fermo/porto-san-giorgio/14269.asp">HOTEL ROSA MEUBLE'</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/ancona/senigallia/17461.asp">ALBERGO MORETTI</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/ancona/sirolo/17921.asp">HOTEL MONTECONERO</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/pesaro-urbino/fano/10415.asp">HOTEL CARAVEL</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/ascoli-piceno/san-benedetto-del-tronto/15892.asp">HOTEL SAYONARA</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/ancona/cupramontana/32035.asp">MULINO BARCHIO</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/marche/pesaro-urbino/pesaro/32059.asp">HOTEL ASTORIA</a></span>, <span class="titololista"><a href="http://www.elenco-alberghi.it/mar

In [31]:
names = [name.text.title() for name in span] # extract the text from each element, change case
print(names)

["Hotel Rosa Meuble'", 'Albergo Moretti', 'Hotel Monteconero', 'Hotel Caravel', 'Hotel Sayonara', 'Mulino Barchio', 'Hotel Astoria', 'Hotel Castello Montegiove', 'Bed And Breakfast Girovagando', 'Casale Del Conero']


Come prima possiamo definire una funzione per rendere il codice riusabile:

In [32]:
def get_names(region, N): # second argument will be provided with our dictionary
    
    print()
    print("Working on "+region+"...")
    print()
    
    region_template_url = "http://www.elenco-alberghi.it/{}/alberghi-hotels_{}.asp"
    
    names = [] # initialize list  
    
    for i in range(1, N + 1):
    
        reg_url = region_template_url.format(region, i) # complete url
        reg_r = requests.get(reg_url, headers=headers) # http request
        reg_soup = bs(reg_r.text, "lxml")  # soupify
        
        # hotel names are in these span elements
        tmp_names = [name.text.title() for name in reg_soup.findAll('span', {'class' : 'titololista'})]
        
        names.extend(tmp_names) # append found names
      
        # andomize the wait between http calls: avoid ip-banning
        timeDelay = 0.1 * random.randrange(0, 3) + 0.5
        time.sleep(timeDelay) # wait some random time
        
    print("Done!")
        
    return(names)

In [33]:
names_molise = get_names("molise", regions_dic["molise"])


Working on molise...

Done!


In [34]:
print(names_molise)

['Dimora Del Prete Di Belmonte', 'B&B Villa Ada', 'Albergo Ristorante Lo Smeraldo', 'Azienda Agrituristica La Ginestra', "Hotel La Fonte Dell'Astore", 'Albergo Le Dune', 'Cascina Garden Hotel', 'Grand Hotel Aljope', 'Grand Hotel Rinascimento', 'Artemide', 'Santo Stefano Dei Cavalli', "Pleiadi'S Hotel ", 'Bar Albergo Hotel La Rondine ', 'Hostel Palazzo Della Citta', 'Hotel Il Duca Del Sannio', 'Masseria Santa Lucia', 'Borgo San Pietro', 'Hotel Majestic Molise', "Hotel Residence L'Airone", 'Albergo Ristorante Miralago', 'Albergo Santoianni', 'Hotel Santa Lucia', 'Masseria Acquasalsa', 'Villaggio Le Meridiane', 'La Romanella', 'Hotel Capodivandra', 'Hotel Di Nardo', 'Masseria Monte Pizzi ', 'Domus Hotel', 'Residence Polena', 'Dimora Spina', 'Hotel Ribo Le Villette', 'B&B La Grotta Delle Fate', 'Agriturismo La Guardata', 'Albergo Campitelli 2', 'San Giorgio Hotel ', 'Hotel Il Cacciatore', 'Aloha Park Hotel ', 'Hotel Kristall', 'Hotel Lo Sciatore', 'Hotel Miletto', 'Albergo Rifugio Iezza', 

Se applichiamo la funzione su tutto il nostro dizionario otterremo le liste di ogni regione:

In [35]:
# this will save one file for each region

for region in regions_dic.keys(): # cycle through region names in our dictionary
    
    with open(region, "w+") as output_file: # write here, one file per region, then we can cat them all together
        
        names = get_names(region, regions_dic[region]) # get results
        
        for name in names: # cycle through names in get_names results
            
            output_file.write(name+"\n") # write to file together with EOL

# Grazie per l'attenzione!